In [1]:
library(tidyverse) # v2.0.0
library(ggridges)  # v0.5.6

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
pathway_abundance <- read.table("PICRUSt2_MetaCyc.txt", sep = "\t", header = TRUE, row.names = 1, check.names = FALSE) # pathway x Samples
pathway_abundance <- pathway_abundance %>%
  rownames_to_column(var = "pathway")


metadata <- read.table("metadata.txt", sep = "\t", header = TRUE, row.names = 1, check.names = FALSE) # Samples x Characteristics
metadata <- metadata %>%
  rownames_to_column(var = "Sample_ID")

annotation <- read.table("PICRUSt2_MetaCyc_descript.tsv", sep = "\t", header = TRUE, check.names = FALSE) # pathway x description

significant_pathways_ids <- c("METH.ACETATE.PWY", "P261.PWY", "PWY.5005", "PWY.6470", "PWY.6906", "PWY.7198", "PWY.7210", "TEICHOICACID.PWY")

pathway_abundance$pathway <- make.names(pathway_abundance$pathway, unique = TRUE)
annotation$pathway <- make.names(annotation$pathway, unique = TRUE)

AbundanceT_melt <- pathway_abundance %>%
  filter(pathway %in% significant_pathways_ids) %>% 
  pivot_longer(cols = -pathway, names_to = "Sample_ID", values_to = "value")

pathway_medians <- AbundanceT_melt %>%
  group_by(pathway) %>%
  
  summarise(median_abundance = median(value[value > 0], na.rm = TRUE)) %>%
  ungroup()

AbundanceT_melt <- AbundanceT_melt %>%
  left_join(pathway_medians, by = "pathway") %>%
  mutate(
    
    median_abundance = ifelse(is.na(median_abundance) | median_abundance == 0, 1e-9, median_abundance),
    
    log10_ratio = log10((value + 1e-9) / median_abundance)
  ) %>%
  dplyr::select(-median_abundance) 

AbundanceT_melt <- AbundanceT_melt %>%
  left_join(metadata %>% dplyr::select(Sample_ID, Group), by = "Sample_ID") 

annotation_sanitized <- annotation %>%
  mutate(pathway_sanitized = make.names(pathway, unique = TRUE))

AbundanceT_melt <- AbundanceT_melt %>%
  left_join(annotation_sanitized %>% dplyr::select(pathway_sanitized, description), 
            by = c("pathway" = "pathway_sanitized"))

AbundanceT_melt <- AbundanceT_melt %>%
  mutate(description = ifelse(is.na(description) | description == "", pathway, description))

AbundanceT_melt <- AbundanceT_melt %>%
  filter(!is.na(Group))

AbundanceT_melt <- AbundanceT_melt %>%
  mutate(
    description = factor(description, levels = rev(unique(description))),
    Group = factor(Group, levels = c("FAP", "Sporadic"))
  )

AbundanceT_melt_for_plot <- AbundanceT_melt %>%
  filter(value > 0)

AbundanceT_melt_for_plot$Group <- factor(AbundanceT_melt_for_plot$Group, levels = c("Sporadic", "FAP"))

group_colors_for_ridges <- c("Sporadic" = "#2200fc", "FAP" = "#590090")

x_axis_min_val <- min(AbundanceT_melt_for_plot$log10_ratio, na.rm = TRUE)
x_axis_max_val <- max(AbundanceT_melt_for_plot$log10_ratio, na.rm = TRUE)

plot_x_min <- floor(x_axis_min_val) -1 
plot_x_max <- ceiling(x_axis_max_val) +1 
if(plot_x_max - plot_x_min < 2) { 
  plot_x_min <- -2
  plot_x_max <- 2
}

if(is.infinite(plot_x_min) | is.infinite(plot_x_max) | is.nan(plot_x_min) | is.nan(plot_x_max) ) {
  plot_x_min <- -5
  plot_x_max <- 5
}

p <- ggplot(AbundanceT_melt_for_plot, 
            aes(x = log10_ratio, ..scaled.., fill = Group, color = Group)) +
  geom_density(alpha = 0.4) +
  geom_jitter(
    aes(y = +0.2 + runif(nrow(AbundanceT_melt_for_plot), 0, 0.1)),
    position = position_jitter(height = 0.2),
    size = 0.8,
    alpha = 0.6,
    show.legend = FALSE
  ) +
  scale_fill_manual(values = group_colors_for_ridges, name = "Group") +
  scale_color_manual(values = group_colors_for_ridges, name = "Group") +
  facet_wrap(~description, ncol = 2) +
  labs(
    x = "Log10(Abundance / Pathway Median)", 
    y = "Density estimate (scaled to maximum of 1.0)",
    title = "Significant Pathway Abundance by Group (FAP vs. Sporadic)"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    axis.title = element_text(face = "bold"),
    axis.text = element_text(color = "black"),
    plot.title = element_text(face = "bold", hjust = 0.5),
    legend.position = "top"
  ) +
  xlim(plot_x_min, plot_x_max) +
  ylim(0, 1.0)

ggsave("Ridge_Plot.pdf", plot = p, width = 12, height = 7)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“number of items read is not a multiple of the number of columns”
Warning message:
“The dot-dot notation (`..scaled..`) was deprecated in ggplot2 3.4.0.
ℹ Please use `after_stat(scaled)` instead.”
